In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#Intliaze data root
data_root = ("/content/drive/MyDrive/MalignantVsBenign/data/")

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root+"train")
batch_size = 32

In [ ]:
#Intialize Training directories
train_ds = tf.keras.utils.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=IMAGE_SHAPE,
  shuffle=True,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=IMAGE_SHAPE,
  batch_size=batch_size)


Found 2637 files belonging to 2 classes.
Using 2110 files for training.
Found 2637 files belonging to 2 classes.
Using 527 files for validation.


In [ ]:
train_ds.class_names

['benign', 'malignant']

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#normalization_layer = layers.Rescaling(1./255)

In [ ]:
data_agumentation = keras.Sequential(
    [
      layers.RandomFlip("horizontal",
                          input_shape=(IMAGE_SHAPE[0],
                                       IMAGE_SHAPE[0],3
                                       )),
     layers.RandomRotation(0.1),
     layers.RandomZoom(0.1),
      
    ]

)

In [ ]:
num_classes = 2

model = Sequential([
      data_agumentation,
      layers.Rescaling(1./255, input_shape=(IMAGE_SHAPE[0],IMAGE_SHAPE[0],3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPool2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPool2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPool2D(),
      layers.Dropout(0.2),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(num_classes)

])

In [ ]:

#Alex Net
num_classes = 2
model = keras.models.Sequential([
    data_agumentation,
    layers.Rescaling(1./255, input_shape=(IMAGE_SHAPE[0],IMAGE_SHAPE[0],3)),
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(IMAGE_SHAPE[0],IMAGE_SHAPE[0],3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])


In [ ]:
#Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),#from_logits=True),
              metrics=['accuracy']
              )

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(
 optimizer=optimizer,
 loss=tf.keras.losses.SparseCategoricalCrossentropy(),
 metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
epochs=200
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/200
66/66 [==============================] - 9s 108ms/step - loss: 1.4900 - acc: 0.7251 - val_loss: 0.6911 - val_acc: 0.5541
Epoch 2/200
66/66 [==============================] - 7s 105ms/step - loss: 1.0800 - acc: 0.7540 - val_loss: 0.8082 - val_acc: 0.4459
Epoch 3/200
66/66 [==============================] - 7s 105ms/step - loss: 0.7558 - acc: 0.7730 - val_loss: 0.6771 - val_acc: 0.5977
Epoch 4/200
66/66 [==============================] - 7s 104ms/step - loss: 0.6311 - acc: 0.7962 - val_loss: 1.4007 - val_acc: 0.5958
Epoch 5/200
66/66 [==============================] - 7s 104ms/step - loss: 0.6263 - acc: 0.7858 - val_loss: 0.4446 - val_acc: 0.7742
Epoch 6/200
66/66 [==============================] - 7s 104ms/step - loss: 0.5176 - acc: 0.7991 - val_loss: 0.4330 - val_acc: 0.7932
Epoch 7/200
66/66 [==============================] - 7s 104ms/step - loss: 0.4913 - acc: 0.8043 - val_loss: 0.6145 - val_acc: 0.7856
Epoch 8/200
66/66 [==============================] - 7s 104ms/step - 

In [ ]:
model.save("/content/drive/MyDrive/MalignantVsBenign/model1")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MalignantVsBenign/model1/assets


In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/MalignantVsBenign/model_AlexNet_200Epoch_87Accuracy')

In [ ]:
TESTING_DATA_DIR = str(data_root+"test")
testing_ds = tf.keras.utils.image_dataset_from_directory(
  TESTING_DATA_DIR,
  subset=None,
  shuffle=True,
  image_size=IMAGE_SHAPE,
  batch_size=batch_size)

Found 660 files belonging to 2 classes.


In [ ]:
final_loss, final_accuracy = new_model.evaluate(testing_ds)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

21/21 [==============================] - 86s 2s/step - loss: 1.2197 - acc: 0.8727
Final loss: 1.22
Final accuracy: 87.27%
